Code adapted from Steven Morse

In [39]:
#hot 100 data set from playlist
import requests
import pandas as pd
from sklearn import preprocessing

CLIENT_ID = 'be31bbd22e5d4a17925cb3f4b999b4cf'
CLIENT_SECRET = '3ca2a7e0b9bd438c82c2e1520b3af534'
AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

data = []

BASE_URL = 'https://api.spotify.com/v1'
PLAYLIST = '/playlists/6UeSakyzhiEt4NB3UAd6NQ/tracks' #BBHot100 as of 11.1.2020

r = requests.get(BASE_URL + PLAYLIST, headers=headers)
tracks = r.json()['items']

for x in tracks:
    f = requests.get(BASE_URL + '/audio-features/' + x['track']['id'], headers=headers)
    f = f.json()
    data.append(f)

df = pd.DataFrame(data)
df = df.drop(columns=['type', 'id','uri','track_href','analysis_url'])
x = df.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled)
df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0.700,0.722,7,-3.558,0,0.0369,0.22100,0.000000,0.2720,0.756,90.989,140526,4
1,0.736,0.802,0,-4.759,1,0.0864,0.46800,0.000000,0.0940,0.675,144.005,172325,4
2,0.554,0.576,6,-6.495,1,0.0399,0.17100,0.000000,0.1590,0.409,74.942,209763,4
3,0.761,0.518,0,-8.871,1,0.1340,0.24400,0.000035,0.1070,0.522,133.976,261493,4
4,0.514,0.730,1,-5.934,1,0.0598,0.00146,0.000095,0.0897,0.334,171.005,200040,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.789,0.711,2,-4.248,1,0.0410,0.03320,0.000000,0.0876,0.753,112.985,190306,4
96,0.711,0.598,3,-7.621,0,0.1820,0.19000,0.000000,0.1500,0.737,140.175,123429,4
97,0.719,0.607,8,-5.156,1,0.1650,0.08000,0.000000,0.1250,0.464,128.006,149900,4
98,0.721,0.632,5,-6.695,0,0.0358,0.06930,0.000000,0.2910,0.296,92.057,144860,4


In [55]:
popularity = []

for x in tracks:
    q = requests.get(BASE_URL + '/tracks/' +  x['track']['id'], headers=headers)
    q = q.json()['popularity']
    popularity.append(q)

mean = sum(popularity)/len(popularity)
print(mean)

for i in range(len(popularity)): 
    if(popularity[i]>=mean):
        popularity[i] = 1
    else: 
        popularity[i] = 0
        
print(popularity)

81.57
[1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1]


In [53]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
import numpy as np

#Gaussian Naive Bayes
X_train, X_test, y_train, y_test = train_test_split(df, popularity, test_size=0.5, random_state=0)
gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)
print("Number of mislabeled points out of a total %d points : %d"
       % (X_test.shape[0], (y_test != y_pred).sum()))
print("Naive Bayes Accuracy:",metrics.accuracy_score(y_test, y_pred))

#K-nearest neighbors
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)

print("KNN Accuracy:",metrics.accuracy_score(y_test, y_pred))

#SVM
clf = svm.SVC(kernel='linear') # Linear Kernel

#Train the model using the training sets
clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

print("SVM Accuracy:",metrics.accuracy_score(y_test, y_pred))

Number of mislabeled points out of a total 50 points : 21
Naive Bayes Accuracy: 0.58
KNN Accuracy: 0.42
SVM Accuracy: 0.52


Utilities

In [111]:
#export df to csv
df.to_csv(index=False)
compression_opts = dict(method='zip',
                        archive_name='out.csv')  
df.to_csv('out.zip', index=False,
          compression=compression_opts)

In [115]:
for col in df.columns: 
    print(col) 

danceability
energy
key
loudness
mode
speechiness
acousticness
instrumentalness
liveness
valence
tempo
type
id
uri
track_href
analysis_url
duration_ms
time_signature


In [112]:
#Old version
import requests
import pandas as pd

CLIENT_ID = 'be31bbd22e5d4a17925cb3f4b999b4cf'
CLIENT_SECRET = '3ca2a7e0b9bd438c82c2e1520b3af534'
AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

# base URL of all Spotify API endpoints
BASE_URL = 'https://api.spotify.com/v1/'

# Track ID from the URI
track_id = ['3AoeaZs8dFemFJr3JdzOL0','6xkryXuiZU360Lngd4sx13','1X3ypzPnyerpZuMqrDFK29',
            '7ySbfLwdCwl1EM0zNCJZ38','3QHONiXGMGU3z68mQInncF','5NaOWLOmVjcA3M5spRATN9',
            '3DUcaEvPO72PijivCjtZcU','33LC84JgLvK2KuW43MfaNq','225xvV8r1yKMHErSWivnow',
            '0uEp9E98JB5awlA084uaIg','2xnVdRMaQBkRfZyefKC2ec','2zZJnAlhgvW6DERn1tu3fb',
            '25ry9EyyRsp1dDctlAclin','2jv9Ppwz9u3lqWfXV0q2sW','6pymOcrCnMuCWdgGVTvUgP',
            '7Ie9W94M7OjPoZVV216Xus','3LUWWox8YYykohBbHUrrxd','7Dyq1bhhjknLKEGg4k5NLr',
            '6r2BECwMgEoRb5yLfp0Hca','1nZzRJbFvCEct3uzu04ZoL','01XFgRZfZI7oBagNf1Loml',
            '5xTtaWoae3wi06K5WfVUUH','4B0JvthVoAAuygILe3n4Bs','4sPmO7WMQUAf45kwMOtONw',
            '0PDUDa38GO8lMxLCRc4lL1','0ZXia0LUmj7r0CK5YOWdTu','7qiZfU4dY1lWllzX7mPBI3',
            '1jYiIOC5d6soxkJP81fxq2','6DCZcSspjsKoFjzjrWoCdn','3CA9pLiwRIGtUBiMjbZmRw',
            '0b9oOr2ZgvyQu88wzixux9','3e9HZxeyfWwjeyPAMmWSSQ','6ocbgoVGwYJhOv1GgI9NsF',
            '22vgEDb5hykfaTwLuskFGD','3eekarcy7kvN4yt5ZFzltW','127QTOFJsJQp5LbJbu3A1y',
            '39Yp9wwQiSRIDOvrVg7mbk','4HBZA5flZLE435QTztThqH','24ySl2hOPGCDcxBxFIqWBu',
            '3dVvWnj4D8JGkKvo6Hucso','4R2kfaDFhslZEMJqAFNpdd','4Oun2ylbjFKMPTiaSbbCih',
            '0t1kP63rueHleOhQkYSXFY','4jVBIpuOiMj1crqd8LoCrJ']

data = []

for i in track_id:
    # actual GET request with proper header
    r = requests.get(BASE_URL + 'audio-features/' + i, headers=headers)
    r = r.json()
    data.append(r)
    

df = pd.DataFrame(data)
df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.651,0.402,11,-9.303,1,0.0280,0.639000,0.000077,0.0765,0.2580,119.873,audio_features,3AoeaZs8dFemFJr3JdzOL0,spotify:track:3AoeaZs8dFemFJr3JdzOL0,https://api.spotify.com/v1/tracks/3AoeaZs8dFem...,https://api.spotify.com/v1/audio-analysis/3Aoe...,345600,4
1,0.671,0.727,7,-7.585,1,0.0350,0.139000,0.000000,0.1230,0.8010,102.358,audio_features,6xkryXuiZU360Lngd4sx13,spotify:track:6xkryXuiZU360Lngd4sx13,https://api.spotify.com/v1/tracks/6xkryXuiZU36...,https://api.spotify.com/v1/audio-analysis/6xkr...,243493,4
2,0.661,0.394,1,-9.032,1,0.0318,0.401000,0.000000,0.1190,0.1070,137.954,audio_features,1X3ypzPnyerpZuMqrDFK29,spotify:track:1X3ypzPnyerpZuMqrDFK29,https://api.spotify.com/v1/tracks/1X3ypzPnyerp...,https://api.spotify.com/v1/audio-analysis/1X3y...,204333,4
3,0.569,0.495,1,-8.964,1,0.0300,0.353000,0.000000,0.0839,0.3020,128.228,audio_features,7ySbfLwdCwl1EM0zNCJZ38,spotify:track:7ySbfLwdCwl1EM0zNCJZ38,https://api.spotify.com/v1/tracks/7ySbfLwdCwl1...,https://api.spotify.com/v1/audio-analysis/7ySb...,281067,4
4,0.862,0.479,7,-9.199,1,0.0646,0.052200,0.001260,0.5890,0.9250,109.878,audio_features,3QHONiXGMGU3z68mQInncF,spotify:track:3QHONiXGMGU3z68mQInncF,https://api.spotify.com/v1/tracks/3QHONiXGMGU3...,https://api.spotify.com/v1/audio-analysis/3QHO...,301520,4
5,0.694,0.823,1,-6.755,1,0.0917,0.118000,0.000007,0.1520,0.6830,95.830,audio_features,5NaOWLOmVjcA3M5spRATN9,spotify:track:5NaOWLOmVjcA3M5spRATN9,https://api.spotify.com/v1/tracks/5NaOWLOmVjcA...,https://api.spotify.com/v1/audio-analysis/5NaO...,299827,4
6,0.450,0.311,4,-8.175,1,0.0314,0.977000,0.007130,0.1670,0.3670,125.648,audio_features,3DUcaEvPO72PijivCjtZcU,spotify:track:3DUcaEvPO72PijivCjtZcU,https://api.spotify.com/v1/tracks/3DUcaEvPO72P...,https://api.spotify.com/v1/audio-analysis/3DUc...,250107,4
7,0.428,0.276,4,-11.729,1,0.0312,0.732000,0.000005,0.1170,0.0382,99.195,audio_features,33LC84JgLvK2KuW43MfaNq,spotify:track:33LC84JgLvK2KuW43MfaNq,https://api.spotify.com/v1/tracks/33LC84JgLvK2...,https://api.spotify.com/v1/audio-analysis/33LC...,280000,1
8,0.383,0.662,2,-5.162,1,0.0333,0.165000,0.000000,0.2480,0.1380,121.154,audio_features,225xvV8r1yKMHErSWivnow,spotify:track:225xvV8r1yKMHErSWivnow,https://api.spotify.com/v1/tracks/225xvV8r1yKM...,https://api.spotify.com/v1/audio-analysis/225x...,298760,4
9,0.535,0.505,2,-8.926,0,0.2450,0.039300,0.000000,0.0923,0.4950,99.935,audio_features,0uEp9E98JB5awlA084uaIg,spotify:track:0uEp9E98JB5awlA084uaIg,https://api.spotify.com/v1/tracks/0uEp9E98JB5a...,https://api.spotify.com/v1/audio-analysis/0uEp...,320267,4
